In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import math
import pandas as pd
import os
from os import path
from tqdm import tqdm
import json
import cv2
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import datetime
import time
import random
from PIL import Image
import pickle
import joblib
import re

from sklearn import preprocessing
import tensorflow as tf
from keras.layers import Input,Dense,LSTM,Flatten,Dropout,concatenate,Conv1D,MaxPooling2D,Activation
from keras.layers import BatchNormalization
from keras.layers import Embedding
from tensorflow.keras import initializers, regularizers
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# import keras_tuner
from tensorflow import keras
import tensorflow_hub as hub
import imgaug.augmenters as iaa
from tensorflow.keras.preprocessing import image, text, sequence
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np

import warnings
warnings.filterwarnings("ignore")

## 1. Error Analysis

In [1]:
#  Reaearch Paper Link: https://arxiv.org/pdf/1505.00468.pdf

def accuracy_metric(X,Y,encoded_features,model):

  predicted_Y = model.predict(encoded_features,verbose=0)
  predicted_class = tf.argmax(predicted_Y, axis=1, output_type=tf.int32)
  predicted_ans = labelencoder.inverse_transform(predicted_class)

  acc_val_lst = []
  for i in tqdm(range(len(Y))):
    acc_val = 0.0
    temp = 0

    for actual_ans in (list(X['answers'])[i]).split(","):
      if actual_ans == predicted_ans[i]:
        temp += 1

    if temp >= 3:
      acc_val = 1
    else:
      acc_val = float(temp)/3

    acc_val_lst.append(acc_val)

  return (sum(acc_val_lst)/len(Y))*100, acc_val_lst

In [2]:
colab_path = "/content/drive/MyDrive/Applied AI/"
test_image= pickle.load(open('/content/drive/MyDrive/Applied AI/CS2/model/test_image_50k_0711.pkl', 'rb'))
tokenizer_50k = pickle.load(open('/content/drive/MyDrive/Applied AI/CS2/model/tokenizer_50k.pkl', 'rb'))
model = tf.keras.models.load_model('/content/drive/MyDrive/Applied AI/CS2/model/model_2lstm_vgg19_50k_1011_50.h5')
labelencoder = pickle.load(open('/content/drive/MyDrive/Applied AI/CS2/model/labelencoder.pkl', 'rb'))

test_sequences = tokenizer_50k.texts_to_sequences(list(X_test['question_preprocessed']))
test_que = pad_sequences(test_sequences, maxlen=22, padding='post')
print(test_image.shape, test_que.shape)

NameError: name 'pickle' is not defined

In [ ]:
Test_accuracy, accuracy_lst = accuracy_metric(X_test,Y_test,[test_que,test_image],model)
print("\nTest Accuracy:",round(Test_accuracy,3))

In [ ]:
test_df = X_test
test_df['answer'] = list(labelencoder.inverse_transform(list(y_test)))
test_df['class_label'] = list(y_test)
test_df['Accuracy'] = list(accuracy_lst)
test_df.head()

In [ ]:
set(test_df['Accuracy'])

In [ ]:
print(f"Only {len(test_df[(test_df.Accuracy != 0) & (test_df.Accuracy != 1)])} datapoints have accuracy 0.33 or 0.66, so add them with correct datapoints.")

In [ ]:
correct_df = test_df[test_df.Accuracy != 0]
incorrect_df = test_df[test_df.Accuracy == 0]
print("Number of Datapoints in correct_df:",len(correct_df))
print("Number of Datapoints in incorrect_df:",len(incorrect_df))

In [ ]:
def top_20_firstword_que(data_f):
  first_word = pd.Series(list(data_f['question_preprocessed'])).apply(lambda x: x.split()[0])
  data_f["que_word"] = list(first_word)
  df = pd.DataFrame((data_f.groupby(['que_word']).count()).index,columns=["que_word"])
  df["que_count"] = list(data_f.groupby(['que_word']).count()['question_preprocessed'])
  df["que%"] = round(df["que_count"]/len(correct_df)*100,2)
  df = df.sort_values(by='que_count',ascending=False)
  return [tuple(x) for x in df.values[:20]]

In [ ]:
def top_20_ans(data_f):
  df = pd.DataFrame((data_f.groupby(['answer']).count()).index,columns=["answer"])
  df["answer_count"] = list(data_f.groupby(['answer']).count()['answers'])
  df["answer%"] = round(df["answer_count"]/len(correct_df)*100,2)
  df = df.sort_values(by='answer_count',ascending=False)
  return [tuple(x) for x in df.values[:20]]

### 1.1 Correct Predicted Datapoints Analysis

In [ ]:
top_20_firstword_que(correct_df)

In [ ]:
top_20_ans(correct_df)

### 1.2 Incorrect Predicted Datapoints Analysis

In [ ]:
top_20_firstword_que(incorrect_df)

In [ ]:
top_20_ans(incorrect_df)

### 1.3 Observations

* In Correctly predicted datapoints, most questions start with the 'is' word and in Incorrectly predicted datapoints, most questions start with the 'what' word.
* In Correctly predicted datapoints, approx 60% of questions have 'yes' or 'no' answers.
* In Incorrectly predicted datapoints, only 20% of questions have 'yes' or 'no' answers and most questions have numeric, color, and other types of answers.
* From this Error analysis, we can conclude that questions that have 'yes' or 'no' answer types are mostly correctly predicted and questions that have numeric or other answer types are a little bit difficult to predict.